In [1]:
# Import libraries
import re
from requests import get
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.chrome.options import Options as SeleniumOptions
from selenium.common.exceptions import ErrorInResponseException, TimeoutException, UnexpectedAlertPresentException
import imdb
import datetime
import pandas as pd


# Getting data from yes planet:


In [2]:
# Launch driver:
driver = webdriver.Chrome(executable_path=r"C:\Users\maxpo\AppData\Roaming\5KPlayer\pack\chromedriver.exe") # path of the chrome driver
Url = r'https://www.yesplanet.co.il/cinemas/rishon_letziyon/1072?lang=en_GB#/buy-tickets-by-cinema?in-cinema=1072&at=2021-11-12&view-mode=list' # <-- fill this
driver.get(Url)
# Get html page source:
page_source = driver.page_source 

In [3]:
#start = '<option class="bs-title-option" value="">Choose a movie</option><option value=""> All Films '
#end = '</option></select></div></div></div>'

pattern = '<option class="bs-title-option" value="">Choose a movie</option><option value=""> All Films (.*?)</option></select></div></div></div>'
page_source_substring = re.search(pattern, page_source).group(1) # Substring of the html source that has the relevant data (movie titles)
#print(page_source_substring)
x = page_source_substring.split("<option id=") # split the substring to a list where each element has a different movie title
x = x[1:] # remove the first element which doesn't contain any movie titles

# Initiate empty list that will hold the movie titles we'll extract from the page source:
movie_titles = [] 
# Go over each element in x and extract the movie title from it:
for string in x:
    title = re.findall(' data-tokens=(.+) data-link=', string)
    movie_titles.append(title[0][1:-1])

#movie_titles
#len(movie_titles)

# Cross referencing movis titles and IMDB scores:

In [4]:
ia = imdb.IMDb() # creating instance of IMDb
outlier_movies = [] #list of movies that either had missing information or title coudln't match imdb movies (usually israeli movies)

rated_movies = [] #lists to hold information for visualization later
ratings = [] #lists to hold information for visualization later

# for each title in movie_titles list input that title in imdb search field and find the appropriate movie: 
for title in movie_titles:
    # searching the title
    search = ia.search_movie(title)
    # loop for printing the title and id of the movie with the latest release year that fit the search,
    # considering the fact we are trying to find rating of movies that are now in the theaters
    for i in range(len(search)):
            if "year" in search[i].data.keys():
                # getting the release year of the movie 
                year = search[i]['year']
                # if release year is the current year (2021): 
                if year == datetime.datetime.now().year:
                    # getting the id of the movie
                    id = search[i].movieID
                    # getting the relevant movie
                    relevant_movie = ia.get_movie(id)
                    if "rating" in relevant_movie.data.keys():
                        # getting rating
                        rating = relevant_movie['rating']
                        # printing it
                        print(search[i]['title'] + ", IMDB rating: ", rating)
                        # update lists:
                        rated_movies.append(search[i]['title'])
                        ratings.append(rating)
                        break
                    else:
                            outlier_movies.append(search[i]['title'])
                            #print("The movie: '" + search[i]['title'] + "' isn't rated")
    
        

print("\n" + "The following movies either had missing information or title coudln't match imdb title (usually israeli movies):")
print(outlier_movies)

Bergman Island, IMDB rating:  6.8
Black Box, IMDB rating:  7.4
The Boss Baby: Family Business, IMDB rating:  5.9
Breaking News in Yuba County, IMDB rating:  5.7
Don't Breathe 2, IMDB rating:  6.1
Dune, IMDB rating:  8.2
Escape Room: Tournament of Champions, IMDB rating:  5.8
Eternals, IMDB rating:  6.9
Halloween Kills, IMDB rating:  5.7
Inheritance, IMDB rating:  7.8
Last Night in Soho, IMDB rating:  7.5
No Time to Die, IMDB rating:  7.5
PAW Patrol: The Movie, IMDB rating:  6.2
Perfect Strangers, IMDB rating:  8.4
Ron's Gone Wrong, IMDB rating:  7.2
Shang-Chi and the Legend of the Ten Rings, IMDB rating:  7.8
Stillwater, IMDB rating:  6.6
The Father Who Moves Mountains, IMDB rating:  5.5
The Last Duel, IMDB rating:  7.7
The World's Most Amazing Vacation Rentals, IMDB rating:  6.7
Venom: Let There Be Carnage, IMDB rating:  6.3

The following movies either had missing information or title coudln't match imdb title (usually israeli movies):
['Such Times Are Happy Times', 'Palm Springs']


# Visualizing the results:

In [5]:
df = pd.DataFrame({ "movie" : rated_movies, "rating" : ratings})
df.sort_values('rating', ascending= False)

,movie,rating
13,Perfect Strangers,8.4
5,Dune,8.2
15,Shang-Chi and the Legend of the Ten Rings,7.8
9,Inheritance,7.8
18,The Last Duel,7.7
10,Last Night in Soho,7.5
11,No Time to Die,7.5
1,Black Box,7.4
14,Ron's Gone Wrong,7.2
7,Eternals,6.9


# code for one title as input for debugging purposes:

In [6]:
ia = imdb.IMDb() # Creating instance of IMDb
title = "Citizen of the World: A Pole-to-Pole Aviation Expedition" # title as input for search field in IMDB

# searching the title
search = ia.search_movie(title)
# loop for printing the title and id of the movie with the latest release year that fit the search,
# considering the fact we are trying to find rating of movies that are now in the theaters
for i in range(len(search)):
		# getting the release year of the movie 
		year = search[i]['year']
		# If release year is the current year (2021): 
		if year == datetime.datetime.now().year:
    		# getting the id of the movie
			id = search[i].movieID
			# getting the relevant movie
			relevant_movie = ia.get_movie(id)
			# Check if the movie has a rating in imdb (sometimes there isn't):
			if "rating" in relevant_movie.data.keys():
				# getting rating
				rating = relevant_movie['rating'] 
				# printing it
				print(search[i]['title'] + ", IMDB rating: ", rating)
				break
			else:
    				print("The movie: '" + search[i]['title'] + "' isn't rated")
    

The movie: 'Citizen of the World: A Pole-to-Pole Aviation Expedition' isn't rated
